# 1. Import necessary libraries for data handling and visualization.

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib.ticker as mticker

# 2. The eSBAE function:

In [2]:
def calculate_areas(db_total, strata_column, categories_column, total_area, z_score):
    
    df_full = db_total.copy()
    df_interpreted = db_total[~db_total[categories_column].isna()]
    
    # get all attributes
    categories = df_interpreted[categories_column].unique()
    
    # get strata
    strata, d = df_interpreted[strata_column].unique(), {}
    print(categories)
    # create stats for each entry
    for category in categories:
        
        if str(category) == 'nan':
            continue
            
        print(f' Calculating stats for {category}')
        # create binary class column
        df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)
        print(f'There are {df_interpreted[category].sum()} entries of {category} in {categories_column}.')
        
        # initialize variables for category <> check the catergories > def, deg, gain
        categories_area, se_total = 0, 0
        d2 = {}
        for stratum in strata:
                        
            if str(stratum) == 'nan':
                continue
            
            # subset to stratum
            df_stratum = df_full[df_full[strata_column] == stratum]
            
            # get area proportion for that stratum on full dataset
            proportion_strata = len(df_stratum)/len(df_full)

            # get stratum area
            stratum_area = proportion_strata * total_area

            # get proportion of forest change within strata from interpreted data
            proportion_category = len(
                df_interpreted[(df_interpreted[strata_column] == stratum) & (df_interpreted[category] == 1)]
            ) / len(
                df_interpreted[df_interpreted[strata_column] == stratum]
            )
            
            # get area from proportion and full area
            category_stratum_area = proportion_category * stratum_area

            # get error from interpreted data for full stratum area
            var = np.var(df_interpreted[category][df_interpreted[strata_column] == stratum])
            sd = np.sqrt(var)               
            n = len(df_interpreted[df_interpreted[strata_column] == stratum])
            se = sd/np.sqrt(n) * stratum_area
                          
            # add for totals
            categories_area += category_stratum_area
            se_total += se**2
            
            # add to dictionary
            d2[f'area_stratum_{stratum}'] = category_stratum_area
            d2[f'ci_stratum_{stratum}']=z_score*se
        
        d2['area_total'] = categories_area
        d2['MOE'] = z_score*np.sqrt(se_total)
        d2['MOE_perc'] =  d2['MOE'] / categories_area * 100
        d[category] = d2
    
    
    return pd.DataFrame.from_dict(d, orient='index')

# 3 Load and prepare interpreted data from various CEO Validation 
#### CIV: EPR 4000 points, 1st Validation 835 points, 2nd Validation 3467 points

In [3]:
pts835 = pd.read_csv('/home/sepal-user/eSBAE_CIV/data/835_final_v2.csv', delimiter=';')
pts3k = pd.read_csv('/home/sepal-user/eSBAE_CIV/data/3K_final_v2.csv', delimiter=';')
erp4k = pd.read_csv('/home/sepal-user/eSBAE_CIV/data/erp4K_final.csv', delimiter=';')

## 3.1 Different data checks and harmonisations 

In [ ]:
#pts835.columns.to_list() 

#### land use IPCC categories 2015 (level 1) = 'ocs_n1_2015'
#### land use national classes 2015 (level 2) = 'ocs_n2_2015'
#### land use IPCC categories 2020 (level 1) = 'ocs_n1_2020'
#### land use national classes 2020 (level 2) = 'ocs_n2_2020'

### Columns of interest => 'point_id', 'ocs_n1_2015', 'ocs_n2_2015', 'ocs_n1_2020', 'ocs_n2_2020', 'kmeans', 'source'

In [ ]:
# verify and count possible values in different columns

#count_values_pts835 = pts835['ocs_n1_2015'].value_counts()
#count_values_pts835 = pts835['ocs_n1_2020'].value_counts()
#count_values_pts835 = pts835['ocs_n2_2015'].value_counts()
#count_values_pts835 = pts835['ocs_n2_2020'].value_counts()
#count_values_pts835 = pts835['kmeans'].value_counts()
#count_values_pts835 = pts835['source'].value_counts()
#print(count_values_pts835)

In [ ]:
#pts3k.columns.to_list()

#### land use IPCC categories 2015 (level 1) = 'ocs_n1_2015'
#### land use national classes 2015 (level 2) = 'ocs_n2_2015'
#### land use IPCC categories 2020 (level 1) = 'ocs_n1_2020'
#### land use national classes 2020 (level 2) = 'ocs_n2_2020'

### Columns of interest => 'point_id', 'ocs_n1_2015', 'ocs_n2_2015', 'ocs_n1_2020', 'ocs_n2_2020', 'kmeans', 'source'

In [ ]:
# verify and count possible values in different columns

#count_values_pts3k = pts3k['ocs_n1_2015'].value_counts()
#count_values_pts3k = pts3k['ocs_n1_2020'].value_counts()
#count_values_pts3k = pts3k['ocs_n2_2015'].value_counts()
#count_values_pts3k = pts3k['ocs_n2_2020'].value_counts()
#count_values_pts3k = pts3k['kmeans'].value_counts()
#count_values_pts3k = pts3k['source'].value_counts()
#print(count_values_pts3k)

In [ ]:
#erp4k.columns 
#erp4k.columns.to_list()
#print(erp4k.head())


In [4]:
# add column 'source'
erp4k['source']= 'CEOERP'

# rename land use categories column 
#### land use IPCC categories 2015 (level 1 or niveau 1) = 'INT1_n1_2015'
#### land use national classes 2015 (level 2 or niveau 2) = 'INT1_n2_2015'
#### land use IPCC categories 2020 (level 1 or niveau 1) = 'INT1_n1_2020'
#### land use national classes 2020 (level 2 or niveau 2) = 'INT1_n2_2020'
erp4k.rename(columns={'INT1_n1_2015':'ocs_n1_2015'}, inplace=True)
erp4k.rename(columns={'INT1_n2_2015':'ocs_n2_2015'}, inplace=True)
erp4k.rename(columns={'INT1_n1_2020':'ocs_n1_2020'}, inplace=True)
erp4k.rename(columns={'INT1_n2_2020':'ocs_n2_2020'}, inplace=True)
erp4k.rename(columns={'LON':'lon'}, inplace=True)
erp4k.rename(columns={'LAT':'lat'}, inplace=True)
erp4k.columns

Index(['point_id', 'PLOTID', 'lon', 'lat', 'chg_prob', 'kmeans', 'str_dal_ne',
       'ID', 'phyto', 'def_year_1520', 'deg_year_1520', 'str_dal_FA',
       'INT1_2000', 'INT1_n1_2000', 'INT1_n2_2000', 'INT1_2010',
       'INT1_n1_2010', 'INT1_n2_2010', 'INT1_2015', 'ocs_n1_2015',
       'ocs_n2_2015', 'INT1_2020', 'ocs_n1_2020', 'ocs_n2_2020', 'INT1_2021',
       'INT1_n1_2021', 'INT1_n2_2021', 'CHG_00_10', 'CHG_10_15', 'CHG_15_20',
       'CHG_20_21', 'Transition_00_10', 'Transition_10_15', 'Transition_15_20',
       'Transition_20_21', 'Gain_Cohort', 'IC', 'COMMENT', 'Wetlands', 'CNC',
       'chg_yr_1520', 'chg_1520', 'gain_1520', 'def_year2015', 'deg_year2015',
       'def_year2016', 'deg_year2016', 'def_year2017', 'deg_year2017',
       'def_year2018', 'deg_year2018', 'def_year2019', 'deg_year2019',
       'def_year2020', 'deg_year2020', 'gain_year2015', 'gain_year2016',
       'gain_year2017', 'gain_year2018', 'gain_year2019', 'gain_year2020',
       'source'],
      dtype='objec

In [ ]:
#duplicated = erp4k['point_id'].duplicated().any() 
#if duplicated:
#    print ("problem")
#else:
#    print ("all good")

In [ ]:
# verify and count possible values in different columns

#count_values_erp4k = erp4k['ocs_n1_2015'].value_counts()
#count_values_erp4k = erp4k['ocs_n1_2020'].value_counts()
#count_values_erp4k = erp4k['ocs_n2_2020'].value_counts()
#count_values_erp4k = erp4k['ocs_n2_2020'].value_counts()
#count_values_erp4k = erp4k['kmeans'].value_counts()
#count_values_erp4k = erp4k['source'].value_counts()
#print(count_values_erp4k)

In [ ]:
#### transition matrix level 1 - 4k pts ERP

#tmatrix = pd.pivot_table(erp4k,values='source',index=['ocs_n1_2015'],columns=['ocs_n1_2020'],aggfunc="count")
#tmatrix

In [ ]:
#### transition matrix level 2 - 4k pts ERP

#tmatrix = pd.pivot_table(erp4k,values='source',index=['ocs_n2_2015'],columns=['ocs_n2_2020'],aggfunc="count")
#tmatrix

# folder path to export data + file name
#file_export = '/home/sepal-user/eSBAE_CIV/data/tmatrixERP.xlsx'
#tmatrix.to_excel(file_export, index=False)

In [ ]:
#### transition matrix level 2 - 4k pts ERP

#tmatrix = pd.pivot_table(erp4k,values='source',index=['ocs_n1_2020'],columns=['ocs_n2_2020'],aggfunc="count")
#tmatrix

## 3.2. Merge all CEO files into a new dataframe = 8300 points

In [5]:
erp4k.dtypes
#pts835.dtypes
#pts3k.dtypes

point_id          object
PLOTID            object
lon              float64
lat              float64
chg_prob          object
                  ...   
gain_year2017      int64
gain_year2018      int64
gain_year2019      int64
gain_year2020      int64
source            object
Length: 62, dtype: object

In [6]:
erp4k_clean = erp4k[erp4k['point_id'] != 'z']
#len(erp4k) # = 3999
#len(erp4k_clean)  #=3998                                              

In [7]:
erp4k_clean['point_id'] = erp4k_clean['point_id'].astype('int64')

/tmp/ipykernel_440/2597961190.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  erp4k_clean['point_id'] = erp4k_clean['point_id'].astype('int64')


In [8]:
##### identifier les colonnes essentielles de CEO
harm_cols = ['point_id', 'ocs_n1_2015', 'ocs_n2_2015', 'ocs_n1_2020', 'ocs_n2_2020', 'kmeans', 'source']

In [9]:
### Concatener les deux tables
#merged_NERP = pd.concat([pts835[harm_cols],pts3k[harm_cols]], axis=0, ignore_index=True)

In [10]:
### Concatener les deux tables
dfconcatCEOfull_db = pd.concat([pts835[harm_cols],pts3k[harm_cols], erp4k_clean[harm_cols]], axis=0, ignore_index=True)
len(dfconcatCEOfull_db) # = 8300

8300

In [ ]:
dfconcatCEOfull_db.columns.to_list()

In [11]:
duplicado = dfconcatCEOfull_db['point_id'].duplicated().any() 
if duplicado:
    print ("problem")
else:
    print ("all good")

problem


In [12]:
doublonsCEO = dfconcatCEOfull_db[dfconcatCEOfull_db.duplicated(subset='point_id', keep=False)]
print(doublonsCEO)

      point_id          ocs_n1_2015                   ocs_n2_2015  \
136     155643     Terres cultivees                      Anacarde   
427      72898   Terres forestieres                   Foret dense   
544     200349  Terres gramineennes                Savane arboree   
624     275438     Terres cultivees          Amenagement agricole   
825     289002     Terres cultivees             Jachere ou fourre   
841        230   Terres forestieres  Foret secondaire ou degradee   
900       2682     Terres cultivees             Jachere ou fourre   
1636     72898   Terres forestieres                   Foret dense   
2389    155643     Terres cultivees                      Anacarde   
2792    200349  Terres gramineennes                Savane arboree   
3603    275438  Terres gramineennes              Savane arbustive   
3792    289002     Terres cultivees                         Hevea   
5515       230   Terres forestieres              Foret secondaire   
5613      2682     Terres cultivee

In [13]:
dfconcatCEOfull_db['doublon'] = 'no'
dfconcatCEOfull_db['doublon'] = np.where((dfconcatCEOfull_db['point_id'] == 155643) & (dfconcatCEOfull_db['source'] == 'CEO3467'), 'yes', dfconcatCEOfull_db['doublon'])
dfconcatCEOfull_db['doublon'] = np.where((dfconcatCEOfull_db['point_id'] == 72898) & (dfconcatCEOfull_db['source'] == 'CEO3467'), 'yes', dfconcatCEOfull_db['doublon'])
dfconcatCEOfull_db['doublon'] = np.where((dfconcatCEOfull_db['point_id'] == 200349) & (dfconcatCEOfull_db['source'] == 'CEO3467'), 'yes', dfconcatCEOfull_db['doublon'])
dfconcatCEOfull_db['doublon'] = np.where((dfconcatCEOfull_db['point_id'] == 275438) & (dfconcatCEOfull_db['source'] == 'CEO3467'), 'yes', dfconcatCEOfull_db['doublon'])
dfconcatCEOfull_db['doublon'] = np.where((dfconcatCEOfull_db['point_id'] == 289002) & (dfconcatCEOfull_db['source'] == 'CEO3467'), 'yes', dfconcatCEOfull_db['doublon'])

count_values_doublon = dfconcatCEOfull_db['doublon'].value_counts()
print(count_values_doublon)

doublon
no     8295
yes       5
Name: count, dtype: int64


In [14]:
dfconcatCEOfull = dfconcatCEOfull_db[dfconcatCEOfull_db['doublon'] == 'no']
len(dfconcatCEOfull)

8295

In [15]:
##create our classes/redd activities/target column that contains the classes for which you want to get the area estimations ('column of interest')
##Create category column == target column == column of interest

dfconcatCEOfull['redd_n1'] = 'problem'

#IPCC categories level
dfconcatCEOfull['redd_n1'] = np.where((dfconcatCEOfull['ocs_n1_2015'] == 'Terres forestieres') & (dfconcatCEOfull['ocs_n1_2020'] != 'Terres forestieres'), 'Def', dfconcatCEOfull['redd_n1'])
dfconcatCEOfull['redd_n1'] = np.where((dfconcatCEOfull['ocs_n1_2015'] != 'Terres forestieres') & (dfconcatCEOfull['ocs_n1_2020'] == 'Terres forestieres'), 'Gain', dfconcatCEOfull['redd_n1'])
dfconcatCEOfull['redd_n1'] = np.where((dfconcatCEOfull['ocs_n1_2015'] == 'Terres forestieres') & (dfconcatCEOfull['ocs_n1_2020'] == 'Terres forestieres'), 'SF', dfconcatCEOfull['redd_n1'])
dfconcatCEOfull['redd_n1'] = np.where((dfconcatCEOfull['ocs_n1_2015'] != 'Terres forestieres') & (dfconcatCEOfull['ocs_n1_2020'] != 'Terres forestieres'), 'SNF', dfconcatCEOfull['redd_n1'])

#new degradation from level 2 (national classes - NERP)
dfconcatCEOfull['redd_n1'] = np.where((dfconcatCEOfull['ocs_n1_2015'] == 'Terres forestieres') & (dfconcatCEOfull['ocs_n2_2020'] == 'Foret secondaire ou degradee'), 'Deg', dfconcatCEOfull['redd_n1'])
#new degradation from level 2 (national classes - ERP)
dfconcatCEOfull['redd_n1'] = np.where((dfconcatCEOfull['ocs_n1_2015'] == 'Terres forestieres') & (dfconcatCEOfull['ocs_n2_2020'] == 'Foret secondaire'), 'Deg', dfconcatCEOfull['redd_n1'])
#deg forêt naturelles --> agroforêt (01/12/23)
dfconcatCEOfull['redd_n1'] = np.where((dfconcatCEOfull['ocs_n2_2015'] == 'Foret secondaire') & (dfconcatCEOfull['ocs_n2_2020'] == 'Plantation agroforestiere'), 'Deg', dfconcatCEOfull['redd_n1'])
dfconcatCEOfull['redd_n1'] = np.where((dfconcatCEOfull['ocs_n2_2015'] == 'Foret dense') & (dfconcatCEOfull['ocs_n2_2020'] == 'Plantation agroforestiere'), 'Deg', dfconcatCEOfull['redd_n1'])

#superficie stable de forêt dégradée (hypothèse dans les calculs sera que pertes et gains sont en équilibre, donc émissions nettes = 0)
dfconcatCEOfull['redd_n1'] = np.where((dfconcatCEOfull['ocs_n2_2015'] == 'Foret secondaire') & (dfconcatCEOfull['ocs_n2_2020'] == 'Foret secondaire'), 'SDegF', dfconcatCEOfull['redd_n1'])
dfconcatCEOfull['redd_n1'] = np.where((dfconcatCEOfull['ocs_n2_2015'] == 'Foret secondaire ou degradee') & (dfconcatCEOfull['ocs_n2_2020'] == 'Foret secondaire ou degradee'), 'SDegF', dfconcatCEOfull['redd_n1'])

count_values_dfconcatCEOfull = dfconcatCEOfull['redd_n1'].value_counts()
print(count_values_dfconcatCEOfull)

redd_n1
SNF      6219
SF       1532
SDegF     301
Def       203
Deg        21
Gain       19
Name: count, dtype: int64


/tmp/ipykernel_440/3600028324.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfconcatCEOfull['redd_n1'] = 'problem'
/tmp/ipykernel_440/3600028324.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfconcatCEOfull['redd_n1'] = np.where((dfconcatCEOfull['ocs_n1_2015'] == 'Terres forestieres') & (dfconcatCEOfull['ocs_n1_2020'] != 'Terres forestieres'), 'Def', dfconcatCEOfull['redd_n1'])
/tmp/ipykernel_440/3600028324.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [ ]:
#df_problem = dfconcatCEOfull[dfconcatCEOfull['redd_n1'] == 'Deg']
#df_problem

In [19]:
# folder path to export data + file name
#file_export = '/home/sepal-user/eSBAE_CIV/data/merged_NERP.xlsx'
#merged_NERP.to_excel(file_export, index=False)

#dfconcatCEOfull.to_csv('/home/sepal-user/eSBAE_CIV/results/dfconcatCEOfull.csv',index=False)

In [20]:
#### transition matrix level 1
tmatrix = pd.pivot_table(dfconcatCEOfull,values='source',index=['redd_n1'],columns=['ocs_n1_2020'],aggfunc="count")
tmatrix

ocs_n1_2020,Autres terres,Etablissement humain,Terres cultivees,Terres forestieres,Terres gramineennes,Terres humides
redd_n1,,,,,,
Def,NaN,4.0,177.0,NaN,21.0,1.0
Deg,NaN,NaN,NaN,21.0,NaN,NaN
Gain,NaN,NaN,NaN,19.0,NaN,NaN
SDegF,NaN,NaN,NaN,301.0,NaN,NaN
SF,NaN,NaN,NaN,1532.0,NaN,NaN
SNF,20.0,98.0,4844.0,NaN,1106.0,151.0


In [ ]:
#### transition matrix level 2

#tmatrix = pd.pivot_table(dfconcatCEOfull,values='source',index=['ocs_n2_2015'],columns=['ocs_n2_2020'],aggfunc="count")
#tmatrix

In [16]:
column_of_interest = 'redd_n1' # your classes/redd activities/target column that contains the classes for which you want to get the area estimations

# 4. Load national grid
##### the country 1km grid csv or gpkg file with all the points and columns including strata or kmeans, chg_prob etc.. 
##### For Ivory Coast we have 325631 points 

In [ ]:
#df320k = gpd.read_file('/home/sepal-user/eSBAE_CIV/data/grid/cote_ivoire_all_classified_20231114.gpkg')
#df320k.to_parquet('/home/sepal-user/module_results/esbae/Cote_Ivoire_MRV/cote_ivoire_all_classified_20231114.parquet')
#df320k = gpd.read_file('/home/sepal-user/eSBAE_CIV/data/grid/320K.csv') ### USE THIS ONE IF NEED TO REPRODUCE

#db_light = df320k[['point_id','images','lon','lat',
#                   'cnc_1520','cnc_2022', 'CNC_ceo', 'FNF_2015', 'FNF_2022', 'database', 
#                   'fnf_prob_2015', 'fnf_prob_2022', 'fnf_max_prob', 'chg_prob',
#                   'simple_combined', 'multiplied', 'kmeans_chgprob', 'kmeans_simcom','kmeans_multi']]
#db_light.to_csv('/home/sepal-user/eSBAE_CIV/data/grid/bdd_320K_noTS.csv',index=False)

In [17]:
df320k = gpd.read_file('/home/sepal-user/eSBAE_CIV/data/grid/bdd_320K_noTS.csv')

print(len(df320k))
print(df320k.columns)

325631
Index(['point_id', 'images', 'lon', 'lat', 'cnc_1520', 'cnc_2022', 'CNC_ceo',
       'FNF_2015', 'FNF_2022', 'database', 'fnf_prob_2015', 'fnf_prob_2022',
       'fnf_max_prob', 'chg_prob', 'simple_combined', 'multiplied',
       'kmeans_chgprob', 'kmeans_simcom', 'kmeans_multi', 'geometry'],
      dtype='object')


In [18]:
## verify column type before merge
#df320k.dtypes
df320k['point_id'] = df320k['point_id'].astype('int64')

In [ ]:
#df320k.head()

In [ ]:
# List all columns 
#df320k.columns.tolist()

##### Import grid with ERP/NERP info and phytogeographic zones = 325,631 pts
###### ERP grid = 46,421 pts
###### NERP grid = 279,210 pts

In [21]:
# import grid with PRE zone (y/n) and phyto zone information (ombrophile, mesophile, sub-soudanais et soudanais)
df320k_erp_phytos = gpd.read_file('/home/sepal-user/eSBAE_CIV/data/grid/GRID_CIV1km_ERP_phytoiffn_phytoCSV.csv') ##csv grid with ERP/NERP + 2 phyto zones
len(df320k_erp_phytos)

325631

In [22]:
#df320k_erp_phytos.dtypes
df320k_erp_phytos['point_id'] = df320k_erp_phytos['point_id'].astype('int64')
#df320k_erp_phytos.dtypes

In [23]:
print(df320k_erp_phytos.columns)

Index(['fid', 'point_id', 'LON', 'PLOTID', 'LAT', 'ERPOBJECTI', 'phytoiffnz',
       'phytoZone', 'geometry'],
      dtype='object')


In [26]:
df320k_zones = df320k[['point_id', 'kmeans_multi', 'lon', 'lat']].merge(df320k_erp_phytos[['point_id', 'ERPOBJECTI', 'phytoiffnz', 'phytoZone']], how='left', on='point_id')
len(df320k_zones)

325631

In [27]:
print(df320k_zones.columns)

Index(['point_id', 'kmeans_multi', 'lon', 'lat', 'ERPOBJECTI', 'phytoiffnz',
       'phytoZone'],
      dtype='object')


In [ ]:
#verif = pd.pivot_table(df320k_zones,values='point_id',index=['ERPOBJECTI'],columns=['phytoZone'],aggfunc="count")
#verif

## 5. FULL dataframe with national GRID (light version) + 8k interpreted points

In [28]:
#count_values = dfconcatCEOfull['redd_n1'].value_counts()
#print(count_values)

redd_n1
SNF      6219
SF       1532
SDegF     301
Def       203
Deg        21
Gain       19
Name: count, dtype: int64


In [29]:
duplicados = df320k_zones['point_id'].duplicated().any() 
if duplicados:
    print ("problem")
else:
    print ("all good")

all good


In [46]:
df320k_full = df320k_zones[['point_id', 'kmeans_multi', 'lon', 'lat', 'ERPOBJECTI', 'phytoiffnz', 'phytoZone']].merge(dfconcatCEOfull[['point_id', 'redd_n1']], how='left', on='point_id')
len(df320k_full)

325633

In [ ]:
count_values = df320k_full['redd_n1'].value_counts()
print(count_values)

In [ ]:
###### préparer catégories par zone phyto (zone iffn et autre):

In [32]:
count_values_phyto_iffn = df320k_full['phytoiffnz'].value_counts()
print(count_values_phyto_iffn)

phytoiffnz
Secteur Mesophile       131577
Secteur SubSoudanais     96018
Secteur Ombrophile       56511
Secteur Soudanais        32396
                          6529
Secteur Montagne          2602
Name: count, dtype: int64


In [48]:
df320k_full['phyto_code_if'] = 'nodata'
df320k_full['phyto_code_if'] = np.where(df320k_full['phytoiffnz'] == 'Secteur Mesophile', 'meso', df320k_full['phyto_code_if'])
df320k_full['phyto_code_if'] = np.where(df320k_full['phytoiffnz'] == 'Secteur SubSoudanais', 'subs', df320k_full['phyto_code_if'])
df320k_full['phyto_code_if'] = np.where(df320k_full['phytoiffnz'] == 'Secteur Ombrophile', 'omb', df320k_full['phyto_code_if'])
df320k_full['phyto_code_if'] = np.where(df320k_full['phytoiffnz'] == 'Secteur Soudanais', 'soud', df320k_full['phyto_code_if'])
df320k_full['phyto_code_if'] = np.where(df320k_full['phytoiffnz'] == 'Secteur Montagne', 'mont', df320k_full['phyto_code_if'])

In [49]:
count_values = df320k_full['phyto_code_if'].value_counts()
print(count_values)

phyto_code_if
meso      131577
subs       96018
omb        56511
soud       32396
nodata      6529
mont        2602
Name: count, dtype: int64


In [50]:
## categories par phytozone IFN
df320k_full['redd_phytoif'] = df320k_full['redd_n1'] + '_' + df320k_full['phyto_code_if']                                                                                                                                                                                                                    

In [51]:
count_values = df320k_full['redd_phytoif'].value_counts()
print(count_values)

redd_phytoif
SNF_omb         2569
SNF_meso        2280
SNF_subs         961
SF_omb           563
SF_subs          530
SF_meso          321
SNF_soud         303
SDegF_omb        135
SDegF_meso       124
SF_soud           79
Def_meso          71
SNF_nodata        70
Def_omb           67
Def_subs          51
SNF_mont          36
SF_nodata         31
SDegF_subs        19
Deg_omb           15
SDegF_mont        11
Gain_omb          11
Def_soud          10
SDegF_nodata       9
Gain_meso          8
SF_mont            8
Deg_meso           5
SDegF_soud         3
Def_mont           3
Deg_subs           1
Def_nodata         1
Name: count, dtype: int64


In [52]:
count_values_phyto2 = df320k_full['phytoZone'].value_counts()
print(count_values_phyto2)

phytoZone
Secteur Soudanais        128809
Secteur mésophile         96599
Secteur Ombrophile        57055
Secteur Sub Soudanais     37819
                           5351
Name: count, dtype: int64


In [53]:
df320k_full['phyto_code'] = 'nodata'
df320k_full['phyto_code'] = np.where(df320k_full['phytoZone'] == 'Secteur Soudanais', 'soud', df320k_full['phyto_code'])
df320k_full['phyto_code'] = np.where(df320k_full['phytoZone'] == 'Secteur mésophile', 'meso', df320k_full['phyto_code'])
df320k_full['phyto_code'] = np.where(df320k_full['phytoZone'] == 'Secteur Ombrophile', 'omb', df320k_full['phyto_code'])
df320k_full['phyto_code'] = np.where(df320k_full['phytoZone'] == 'Secteur Sub Soudanais', 'subs', df320k_full['phyto_code'])


In [54]:
count_values = df320k_full['phyto_code'].value_counts()
print(count_values)

phyto_code
soud      128809
meso       96599
omb        57055
subs       37819
nodata      5351
Name: count, dtype: int64


In [56]:
## categories par phytozone
df320k_full['redd_phyto'] = df320k_full['redd_n1'] + '_' + df320k_full['phyto_code'] 

In [57]:
count_values2 = df320k_full['redd_phyto'].value_counts()
print(count_values2)

redd_phyto
SNF_omb         2571
SNF_meso        1947
SNF_soud        1268
SF_soud          612
SF_omb           566
SNF_subs         374
SF_meso          239
SDegF_omb        135
SDegF_meso       117
SF_subs           91
Def_meso          68
Def_omb           67
Def_soud          61
SNF_nodata        59
SF_nodata         24
SDegF_soud        22
SDegF_subs        18
Deg_omb           15
Gain_omb          11
SDegF_nodata       9
Gain_meso          6
Def_subs           6
Deg_meso           4
Gain_subs          2
Deg_subs           1
Deg_soud           1
Def_nodata         1
Name: count, dtype: int64


In [58]:
# Scenario DA 1 :
df_scDA1 = df320k_full[df320k_full['ERPOBJECTI'] != '1']
len(df_scDA1)

279210

In [59]:
count_values = df_scDA1['redd_n1'].value_counts()
print(count_values)

redd_n1
SNF      3096
SF        898
SDegF     176
Def       100
Gain        4
Deg         2
Name: count, dtype: int64


In [60]:
df320k_full.to_csv('/home/sepal-user/eSBAE_CIV/results/df320k_full.csv',index=False)

In [ ]:
# Scenario DA 3 : UTILISER df320k_full
#df_sc_DA3 = df320k_full
#len(df_scDA3)

##### Perform area calculation using the stratum column. In this case the column is called kmeans. Use the merge dataframe (national grid points + CEO validated points)

## 6. Run the eSBAE function for the different scenarios

In [61]:
#scenario DA1: NERF = ERP + NERP
calculate_areas(db_total=df_scDA1, strata_column='kmeans_multi', categories_column='redd_n1', total_area=len(df_scDA1), z_score=1.645)

['SNF' 'SF' 'SDegF' 'Def' 'Deg' 'Gain']
 Calculating stats for SNF
There are 3096 entries of SNF in redd_n1.
 Calculating stats for SF
There are 898 entries of SF in redd_n1.


/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)
/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)


 Calculating stats for SDegF
There are 176 entries of SDegF in redd_n1.
 Calculating stats for Def
There are 100 entries of Def in redd_n1.


/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)
/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)


 Calculating stats for Deg
There are 2 entries of Deg in redd_n1.
 Calculating stats for Gain
There are 4 entries of Gain in redd_n1.


/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)
/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)


,area_stratum_0,ci_stratum_0,area_stratum_2,ci_stratum_2,area_stratum_1,ci_stratum_1,area_total,MOE,MOE_perc
SNF,57362.060818,1407.447738,148446.064286,2426.288709,15022.431034,756.949867,220830.556138,2905.298459,1.315623
SF,20926.606181,1323.334425,17379.051429,2256.532155,7777.793103,686.960893,46083.450713,2704.637986,5.869001
SDegF,2357.344965,506.281514,1206.878571,625.565603,3418.465517,503.751740,6982.689054,949.430940,13.596924
Def,2274.631107,497.575172,1568.942143,712.484413,1003.586207,286.423060,4847.159456,915.015442,18.877354
Deg,0.000000,0.000000,120.687857,198.460608,31.362069,51.560945,152.049926,205.049126,134.856446
Gain,41.356929,68.015189,241.375714,280.565356,31.362069,51.560945,314.094712,293.260151,93.366790


In [63]:
#scenario DA1 par phyto IFFN: 
calculate_areas(db_total=df_scDA1, strata_column='kmeans_multi', categories_column='redd_phytoif', total_area=len(df_scDA1), z_score=1.645)

['SNF_subs' 'SF_subs' 'SF_nodata' 'SDegF_meso' 'SNF_meso' 'SNF_omb'
 'Def_omb' 'SF_meso' 'SDegF_omb' 'SDegF_subs' 'Def_meso' 'SNF_soud'
 'SF_omb' 'SNF_nodata' 'SF_soud' 'SDegF_nodata' 'Def_subs' 'Def_soud'
 'SDegF_soud' 'Deg_meso' 'Deg_subs' 'SNF_mont' 'SDegF_mont' 'SF_mont'
 'Def_nodata' 'Def_mont' 'Gain_meso']
 Calculating stats for SNF_subs
There are 961 entries of SNF_subs in redd_phytoif.
 Calculating stats for SF_subs
There are 530 entries of SF_subs in redd_phytoif.


/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)
/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)


 Calculating stats for SF_nodata
There are 29 entries of SF_nodata in redd_phytoif.
 Calculating stats for SDegF_meso
There are 95 entries of SDegF_meso in redd_phytoif.


/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)
/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)


 Calculating stats for SNF_meso
There are 1436 entries of SNF_meso in redd_phytoif.
 Calculating stats for SNF_omb
There are 305 entries of SNF_omb in redd_phytoif.


...

 Calculating stats for Def_nodata
There are 1 entries of Def_nodata in redd_phytoif.
 Calculating stats for Def_mont
There are 3 entries of Def_mont in redd_phytoif.


/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)
/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)


 Calculating stats for Gain_meso
There are 4 entries of Gain_meso in redd_phytoif.


/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)


,area_stratum_0,ci_stratum_0,area_stratum_2,ci_stratum_2,area_stratum_1,ci_stratum_1,area_total,MOE,MOE_perc
SNF_subs,19520.470588,1292.505505,50326.836429,3397.114986,2258.068966,419.255410,72105.375982,3658.788844,5.074225
SF_subs,13813.214357,1135.115600,10258.467857,1773.935398,3481.189655,507.683862,27552.871869,2166.351109,7.862524
SF_nodata,827.138584,302.728523,482.751429,396.495411,156.810345,115.028124,1466.700358,511.942027,34.904336
SDegF_meso,1075.280160,344.641833,844.815000,523.950234,1944.448276,391.482638,3864.543435,739.297300,19.130262
SNF_meso,24690.086740,1393.129166,63361.125000,3596.243906,9847.689655,730.824674,97898.901395,3925.287736,4.009532
SNF_omb,6658.465603,827.865617,10741.219286,1812.432369,1724.913793,370.314852,19124.598682,2026.673546,10.597208
Def_omb,82.713858,96.164013,0.000000,0.000000,0.000000,0.000000,82.713858,96.164013,116.261065
SF_meso,3391.268195,603.334531,3258.572143,1021.604063,3355.741379,499.764196,10005.581718,1287.420549,12.867023
SDegF_omb,661.710867,271.041163,120.687857,198.460608,689.965517,238.902254,1472.364242,412.218646,27.997056
SDegF_subs,289.498504,179.681821,241.375714,280.565356,313.620690,162.203495,844.494908,370.556945,43.879121


In [64]:
#scenario DA1 par phyto : 
calculate_areas(db_total=df_scDA1, strata_column='kmeans_multi', categories_column='redd_phyto', total_area=len(df_scDA1), z_score=1.645)

['SNF_soud' 'SF_soud' 'SDegF_meso' 'SNF_meso' 'SNF_omb' 'Def_omb'
 'SF_subs' 'SDegF_omb' 'SF_meso' 'SNF_subs' 'SDegF_soud' 'Def_meso'
 'SF_omb' 'SNF_nodata' 'SDegF_subs' 'SDegF_nodata' 'SF_nodata' 'Def_soud'
 'Def_subs' 'Deg_subs' 'Deg_soud' 'Def_nodata' 'Gain_meso' 'Gain_subs']
 Calculating stats for SNF_soud
There are 1268 entries of SNF_soud in redd_phyto.
 Calculating stats for SF_soud
There are 612 entries of SF_soud in redd_phyto.


/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)
/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)


 Calculating stats for SDegF_meso
There are 88 entries of SDegF_meso in redd_phyto.
 Calculating stats for SNF_meso
There are 1107 entries of SNF_meso in redd_phyto.


/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)
/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)


 Calculating stats for SNF_omb
There are 306 entries of SNF_omb in redd_phyto.
 Calculating stats for Def_omb
There are 2 entries of Def_omb in redd_phyto.


...

 Calculating stats for Deg_soud
There are 1 entries of Deg_soud in redd_phyto.
 Calculating stats for Def_nodata
There are 1 entries of Def_nodata in redd_phyto.


/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)
/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)


 Calculating stats for Gain_meso
There are 2 entries of Gain_meso in redd_phyto.
 Calculating stats for Gain_subs
There are 2 entries of Gain_subs in redd_phyto.


/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)
/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)


,area_stratum_0,ci_stratum_0,area_stratum_2,ci_stratum_2,area_stratum_1,ci_stratum_1,area_total,MOE,MOE_perc
SNF_soud,24855.514457,1395.802978,70602.396429,3663.719349,2571.689655,444.611832,98029.600540,3945.729984,4.025039
SF_soud,16087.845464,1204.698954,12913.600714,1973.588528,3638.000000,517.279954,32639.446178,2369.373292,7.259233
SDegF_meso,868.495513,310.126320,603.439286,443.136544,1944.448276,391.482638,3416.383075,667.687792,19.543704
SNF_meso,19561.827517,1293.452160,40792.495714,3178.967363,9283.172414,720.962678,69637.495646,3506.941569,5.035996
SNF_omb,6658.465603,827.865617,10861.907143,1821.890886,1724.913793,370.314852,19245.286539,2035.136597,10.574727
Def_omb,82.713858,96.164013,0.000000,0.000000,0.000000,0.000000,82.713858,96.164013,116.261065
SF_subs,1736.991027,436.258709,1931.005714,789.575197,1034.948276,290.690394,4702.945017,947.761340,20.152507
SDegF_omb,661.710867,271.041163,120.687857,198.460608,689.965517,238.902254,1472.364242,412.218646,27.997056
SF_meso,1861.061815,451.225647,1327.566429,655.862690,2446.241379,434.731264,5634.869622,907.056627,16.097207
SNF_subs,5665.899302,768.623497,23292.756429,2560.930251,1254.482759,318.698503,30213.138489,2692.715092,8.912398


In [65]:
#scenario DA3: NERF = ERP + NERP
calculate_areas(db_total=df320k_full, strata_column='kmeans_multi', categories_column='redd_n1', total_area=len(df320k_full), z_score=1.645)

['SNF' 'SF' 'SDegF' 'Def' 'Gain' 'Deg']
 Calculating stats for SNF
There are 6219 entries of SNF in redd_n1.


/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)


 Calculating stats for SF
There are 1532 entries of SF in redd_n1.
 Calculating stats for SDegF
There are 301 entries of SDegF in redd_n1.


/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)
/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)


 Calculating stats for Def
There are 203 entries of Def in redd_n1.
 Calculating stats for Gain
There are 19 entries of Gain in redd_n1.


/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)
/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)


 Calculating stats for Deg
There are 21 entries of Deg in redd_n1.


/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)


,area_stratum_0,ci_stratum_0,area_stratum_2,ci_stratum_2,area_stratum_1,ci_stratum_1,area_total,MOE,MOE_perc
SNF,67551.291848,1306.574214,177170.739719,1549.800593,15534.419417,712.429539,260256.450985,2148.621396,0.825579
SF,22559.763502,1221.268139,17009.639427,1478.698042,11064.401294,669.426225,50633.804224,2031.298904,4.011745
SDegF,2943.969138,497.047650,884.293181,352.010861,4248.730097,472.018573,8076.992416,770.564433,9.540240
Def,1983.979202,410.150487,780.258689,330.744745,2788.229126,391.594015,5552.467017,656.476489,11.823150
Gain,319.996645,166.181114,156.051738,148.149767,132.772816,88.992726,608.821199,239.758673,39.380802
Deg,31.999665,52.630618,52.017246,85.557017,420.447249,157.693445,504.464160,186.968468,37.062785


In [66]:
#scenario DA3: NERF = ERP + NERP par phyto IFFN
calculate_areas(db_total=df320k_full, strata_column='kmeans_multi', categories_column='redd_phytoif', total_area=len(df320k_full), z_score=1.645)

['SNF_subs' 'SF_subs' 'SF_nodata' 'SDegF_meso' 'SNF_meso' 'SNF_omb'
 'Def_omb' 'SF_meso' 'SDegF_omb' 'SDegF_subs' 'Def_meso' 'SNF_soud'
 'SF_omb' 'SNF_nodata' 'SF_soud' 'SDegF_nodata' 'Gain_omb' 'Def_subs'
 'Gain_meso' 'Def_soud' 'SDegF_soud' 'Deg_meso' 'Deg_subs' 'Deg_omb'
 'SNF_mont' 'SDegF_mont' 'SF_mont' 'Def_nodata' 'Def_mont']
 Calculating stats for SNF_subs
There are 961 entries of SNF_subs in redd_phytoif.
 Calculating stats for SF_subs
There are 530 entries of SF_subs in redd_phytoif.


/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)
/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)


 Calculating stats for SF_nodata
There are 31 entries of SF_nodata in redd_phytoif.
 Calculating stats for SDegF_meso
There are 124 entries of SDegF_meso in redd_phytoif.


/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)
/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)


 Calculating stats for SNF_meso
There are 2280 entries of SNF_meso in redd_phytoif.
 Calculating stats for SNF_omb
There are 2569 entries of SNF_omb in redd_phytoif.


...

 Calculating stats for SF_mont
There are 8 entries of SF_mont in redd_phytoif.
 Calculating stats for Def_nodata
There are 1 entries of Def_nodata in redd_phytoif.


/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)
/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)


 Calculating stats for Def_mont
There are 3 entries of Def_mont in redd_phytoif.


/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)


,area_stratum_0,ci_stratum_0,area_stratum_2,ci_stratum_2,area_stratum_1,ci_stratum_1,area_total,MOE,MOE_perc
SNF_subs,15103.841664,1049.184026,21691.191563,1647.859655,1593.273786,301.597124,38388.307013,1976.661172,5.149123
SF_subs,10687.887957,906.526583,4421.465906,779.954848,2456.297087,369.484003,17565.650951,1251.654281,7.125579
SF_nodata,639.993291,234.619736,260.086230,191.209734,132.772816,88.992726,1032.852336,315.479458,30.544488
SDegF_meso,1023.989265,296.171125,416.137968,241.766904,1858.819417,324.432480,3298.946650,501.422980,15.199487
SNF_meso,26207.725260,1282.921654,55034.246219,2360.520525,8917.907443,628.268304,90159.878923,2759.106047,3.060237
SNF_omb,21087.778933,1192.625675,89521.680287,2616.710034,4182.343689,468.835320,114791.802909,2913.646162,2.538201
Def_omb,255.997316,148.686928,104.034492,120.979837,1261.341748,269.710881,1621.373556,330.889533,20.407976
SF_meso,3423.964106,534.645124,1404.465641,443.030843,4138.086084,466.691900,8966.515831,836.614048,9.330425
SDegF_omb,1439.984904,350.440767,364.120722,226.182294,1836.690615,322.605901,3640.796241,527.296623,14.483003
SDegF_subs,223.997652,139.107275,104.034492,120.979837,221.288026,114.739714,549.320170,217.145474,39.529856


In [62]:
#scenario DA3: NERF = ERP + NERP par phyto 
calculate_areas(db_total=df320k_full, strata_column='kmeans_multi', categories_column='redd_phyto', total_area=len(df320k_full), z_score=1.645)

['SNF_soud' 'SF_soud' 'SDegF_meso' 'SNF_meso' 'SNF_omb' 'Def_omb'
 'SF_subs' 'SDegF_omb' 'SF_meso' 'SNF_subs' 'SDegF_soud' 'Def_meso'
 'SF_omb' 'SNF_nodata' 'SDegF_subs' 'SDegF_nodata' 'SF_nodata' 'Gain_omb'
 'Def_soud' 'Gain_meso' 'Def_subs' 'Deg_subs' 'Deg_soud' 'Deg_omb'
 'Deg_meso' 'Def_nodata' 'Gain_subs']
 Calculating stats for SNF_soud
There are 1268 entries of SNF_soud in redd_phyto.
 Calculating stats for SF_soud
There are 612 entries of SF_soud in redd_phyto.


/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)
/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)


 Calculating stats for SDegF_meso
There are 117 entries of SDegF_meso in redd_phyto.
 Calculating stats for SNF_meso
There are 1947 entries of SNF_meso in redd_phyto.


/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)
/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)


 Calculating stats for SNF_omb
There are 2571 entries of SNF_omb in redd_phyto.
 Calculating stats for Def_omb
There are 67 entries of Def_omb in redd_phyto.


...

 Calculating stats for Deg_meso
There are 4 entries of Deg_meso in redd_phyto.
 Calculating stats for Def_nodata
There are 1 entries of Def_nodata in redd_phyto.


/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)
/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)


 Calculating stats for Gain_subs
There are 2 entries of Gain_subs in redd_phyto.


/tmp/ipykernel_440/804014814.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interpreted[category] =  df_interpreted[categories_column].apply(lambda x: 1 if x == category else 0)


,area_stratum_0,ci_stratum_0,area_stratum_2,ci_stratum_2,area_stratum_1,ci_stratum_1,area_total,MOE,MOE_perc
SNF_soud,19231.798390,1153.071012,30430.088883,1902.237115,1814.561812,320.766248,51476.449085,2247.436270,4.365950
SF_soud,12447.869507,968.105392,5565.845317,872.471496,2566.941100,377.054982,20580.655924,1356.688992,6.592059
SDegF_meso,863.990943,272.281082,312.103476,209.431943,1858.819417,324.432480,3034.913836,472.498847,15.568773
SNF_meso,22239.766857,1215.236640,45098.952242,2210.845740,8519.588997,618.897793,75858.308095,2597.628429,3.424316
SNF_omb,21119.778598,1193.273175,89573.697533,2616.831044,4182.343689,468.835320,114875.819820,2914.019928,2.536670
Def_omb,255.997316,148.686928,104.034492,120.979837,1261.341748,269.710881,1621.373556,330.889533,20.407976
SF_subs,1343.985911,338.730870,832.275935,341.546203,730.250485,206.867583,2906.512331,523.628311,18.015692
SDegF_omb,1439.984904,350.440767,364.120722,226.182294,1836.690615,322.605901,3640.796241,527.296623,14.483003
SF_meso,2239.976518,435.211600,572.189705,283.383734,3496.350809,433.537181,6308.517033,676.513093,10.723805
SNF_subs,4383.954042,601.804484,10247.397453,1169.200592,885.152104,227.225894,15516.503599,1334.477526,8.600375


### 7.  Résultats ]2020-2022]

In [ ]:
pts835_22 = pd.read_csv('/home/sepal-user/eSBAE_CIV/data/835_final_v3_2022.csv', delimiter=';')
pts835_22.rename(columns={'oce_n1_2022':'ocs_n1_2022'}, inplace=True)
pts835_22.columns.to_list()

In [ ]:
pts3k_22 = pd.read_csv('/home/sepal-user/eSBAE_CIV/data/3K_final_v3_2022.csv', delimiter=';')
pts3k_22.columns.to_list()

In [ ]:
##### identifier les colonnes essentielles de CEO
harm_cols22 = ['point_id', 'ocs_n1_2020', 'ocs_n2_2020', 'ocs_n1_2022', 'ocs_n2_2022','source']

In [ ]:
### Concatener les deux tables
dfconcatCEO22 = pd.concat([pts835_22[harm_cols22],pts3k_22[harm_cols22]], axis=0, ignore_index=True)
len(dfconcatCEO22) # = 4302                    


In [ ]:
duplicado22 = dfconcatCEO22['point_id'].duplicated().any() 
if duplicado22:
    print ("problem")
else:
    print ("all good")

In [ ]:
doublonsCEO22 = dfconcatCEO22[dfconcatCEO22.duplicated(subset='point_id', keep=False)]
print(doublonsCEO22)

In [ ]:
dfconcatCEO22['doublon'] = 'no'
dfconcatCEO22['doublon'] = np.where((dfconcatCEO22['point_id'] == 155643) & (dfconcatCEO22['source'] == 'CEO3467'), 'yes', dfconcatCEO22['doublon'])
dfconcatCEO22['doublon'] = np.where((dfconcatCEO22['point_id'] == 72898) & (dfconcatCEO22['source'] == 'CEO3467'), 'yes', dfconcatCEO22['doublon'])
dfconcatCEO22['doublon'] = np.where((dfconcatCEO22['point_id'] == 200349) & (dfconcatCEO22['source'] == 'CEO3467'), 'yes', dfconcatCEO22['doublon'])
dfconcatCEO22['doublon'] = np.where((dfconcatCEO22['point_id'] == 275438) & (dfconcatCEO22['source'] == 'CEO3467'), 'yes', dfconcatCEO22['doublon'])
dfconcatCEO22['doublon'] = np.where((dfconcatCEO22['point_id'] == 289002) & (dfconcatCEO22['source'] == 'CEO3467'), 'yes', dfconcatCEO22['doublon'])

count_values_doublon22 = dfconcatCEO22['doublon'].value_counts()
print(count_values_doublon22)

In [ ]:
dfconcatCEO22_clean = dfconcatCEO22[dfconcatCEO22['doublon'] == 'no']
len(dfconcatCEO22_clean)


In [ ]:
#dfconcatCEO22_clean.to_csv('/home/sepal-user/eSBAE_CIV/data/dfconcatCEO22_clean.csv',index=False)

In [ ]:
#### transition matrix level 1
tmatrix2022 = pd.pivot_table(dfconcatCEO22_clean,values='source',index=['ocs_n1_2020'],columns=['ocs_n1_2022'],aggfunc="count")
tmatrix2022

In [ ]:
#### transition matrix level 1
tmatrix2022n2 = pd.pivot_table(dfconcatCEO22_clean,values='source',index=['ocs_n2_2020'],columns=['ocs_n2_2022'],aggfunc="count")
tmatrix2022n2


In [ ]:
##create our classes/redd activities/target column that contains the classes for which you want to get the area estimations ('column of interest')
##Create category column == target column == column of interest

dfconcatCEO22_clean['redd_22'] = 'problem'

#IPCC categories level
dfconcatCEO22_clean['redd_22'] = np.where((dfconcatCEO22_clean['ocs_n1_2020'] == 'Terres forestieres') & (dfconcatCEO22_clean['ocs_n1_2022'] != 'Terres forestieres'), 'Def', dfconcatCEO22_clean['redd_22'])
dfconcatCEO22_clean['redd_22'] = np.where((dfconcatCEO22_clean['ocs_n1_2020'] != 'Terres forestieres') & (dfconcatCEO22_clean['ocs_n1_2022'] == 'Terres forestieres'), 'Gain', dfconcatCEO22_clean['redd_22'])
dfconcatCEO22_clean['redd_22'] = np.where((dfconcatCEO22_clean['ocs_n1_2020'] == 'Terres forestieres') & (dfconcatCEO22_clean['ocs_n1_2022'] == 'Terres forestieres'), 'SF', dfconcatCEO22_clean['redd_22'])
dfconcatCEO22_clean['redd_22'] = np.where((dfconcatCEO22_clean['ocs_n1_2020'] != 'Terres forestieres') & (dfconcatCEO22_clean['ocs_n1_2022'] != 'Terres forestieres'), 'SNF', dfconcatCEO22_clean['redd_22'])

#new degradation from level 2 (national classes - NERP)
dfconcatCEO22_clean['redd_22'] = np.where((dfconcatCEO22_clean['ocs_n1_2020'] == 'Terres forestieres') & (dfconcatCEO22_clean['ocs_n2_2022'] == 'Foret secondaire ou degradee'), 'Deg', dfconcatCEO22_clean['redd_22'])
#new degradation from level 2 (national classes - ERP)
dfconcatCEO22_clean['redd_22'] = np.where((dfconcatCEO22_clean['ocs_n1_2020'] == 'Terres forestieres') & (dfconcatCEO22_clean['ocs_n2_2022'] == 'Foret secondaire'), 'Deg', dfconcatCEO22_clean['redd_22'])

#superficie stable de forêt dégradée (hypothèse dans les calculs sera que pertes et gains sont en équilibre, donc émissions nettes = 0)
dfconcatCEO22_clean['redd_22'] = np.where((dfconcatCEO22_clean['ocs_n2_2020'] == 'Foret secondaire') & (dfconcatCEO22_clean['ocs_n2_2022'] == 'Foret secondaire'), 'SDegF', dfconcatCEO22_clean['redd_22'])
dfconcatCEO22_clean['redd_22'] = np.where((dfconcatCEO22_clean['ocs_n2_2020'] == 'Foret secondaire ou degradee') & (dfconcatCEO22_clean['ocs_n2_2022'] == 'Foret secondaire ou degradee'), 'SDegF', dfconcatCEO22_clean['redd_22'])

count_values_dfconcatCEO22 = dfconcatCEO22_clean['redd_22'].value_counts()
print(count_values_dfconcatCEO22)

In [ ]:
df320k_full_NERP22 = df320k_full[['point_id', 'kmeans_multi', 'LON', 'LAT', 'ERPOBJECTI', 'phytoiffnz', 'phytoZone']].merge(dfconcatCEO22_clean[['point_id', 'redd_22']], how='left', on='point_id')
len(df320k_full_NERP22)

In [ ]:
#resultats 2022: NERP
calculate_areas(db_total=df320k_full_NERP22, strata_column='kmeans_multi', categories_column='redd_22', total_area=len(df320k_full_NERP22), z_score=1.645)